In [ ]:
import os

import torch
from fastbook import *

In [ ]:

path = Path('/mnist')
path.ls()

In [ ]:
(path/"training").ls()

In [ ]:
(path/"testing").ls()

In [ ]:
training_ones = (path / "training" / "1").ls().sorted()
training_fives = (path / "training" / "5").ls().sorted()
testing_ones = (path / "testing" / "1").ls().sorted()
testing_fives = (path / "testing" / "5").ls().sorted()
training_ones, training_fives, testing_ones, testing_fives

In [ ]:
import PIL.Image

img_1 = PIL.Image.open(training_ones[0])
img_1

In [ ]:
tensor(img_1)

In [ ]:
training_ones_tensor = torch.stack([tensor(Image.open(img)) for img in training_ones]).float()/255
testing_ones_tensor = torch.stack([tensor(Image.open(img)) for img in testing_ones]).float()/255
training_ones_tensor.shape, testing_ones_tensor.shape

In [ ]:
training_fives_tensor = torch.stack([tensor(Image.open(img)) for img in training_fives]).float()/255
testing_fives_tensor = torch.stack([tensor(Image.open(img)) for img in testing_fives]).float()/255
training_fives_tensor.shape, testing_fives_tensor.shape

In [ ]:
train_x = torch.cat([
    torch.reshape(training_ones_tensor, (len(training_ones_tensor), 28*28)),
    torch.reshape(training_fives_tensor, (len(training_fives_tensor), 28*28)),
    ])
train_x.shape

In [ ]:
train_y = torch.cat([torch.full((len(training_ones_tensor),), 1), torch.full((len(training_fives_tensor),), 0)])
train_y.shape

In [ ]:
valid_x = torch.cat([
    torch.reshape(testing_ones_tensor, (len(testing_ones_tensor), 28*28)),
    torch.reshape(testing_fives_tensor, (len(testing_fives_tensor), 28*28)), 
])
valid_x.shape

In [ ]:
valid_y = torch.cat([torch.full((len(testing_ones_tensor),), 1), torch.full((len(testing_fives_tensor),), 0)])
valid_y.shape

In [ ]:
def init_params(size, std=1.0):
    return (torch.randn(size) * std).requires_grad_()

In [ ]:
def loss(pred: Tensor, target: Tensor):
    pred = pred.sigmoid()
    return torch.where(target==1, 1 - pred, pred).mean()
    

In [ ]:
dl = DataLoader(list(zip(train_x, train_y)), batch_size=256)
xb, yb = first(dl)
xb.shape, yb.shape 

In [ ]:
valid_dl = DataLoader(list(zip(valid_x, valid_y)))

In [ ]:
def calc_grad(xb, yb, model):
    preds = model(xb)
    l = loss(preds, yb)
    l.backward()

class BasicOptim:
    
    def __init__(self, params: Tensor, lr: float):
        self.params = list(params)
        self.lr = lr
        
    def step(self):
        for p in self.params:
            # print(f'gradient: {p.grad.data[4:4:4]}')
            p.data -= p.grad.data * self.lr
            
    def zero_grad(self):
        for p in self.params:
            p.grad = None

def train_epoch(model, opt: BasicOptim):
    for xb, yb in dl:
        calc_grad(xb, yb, model)
        opt.step()
        opt.zero_grad()
        
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()

def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb,yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)


lr = 1.

linear_model = nn.Linear(28 * 28, 1)

opt = BasicOptim(linear_model.parameters(), lr)

for i in range(20):
    train_epoch(linear_model, opt)
    print(validate_epoch(linear_model))

In [ ]:
linear_model(valid_x)